In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import tensorflow.keras
import os
checkpoint_path = "training_ResNet_52/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
from keras.callbacks import History 
from numba import cuda
from time import time
from pprint import pprint
%autosave 2
import gc

Autosaving every 2 seconds


In [2]:
def mem_clr():
    device=cuda.get_current_device()
    device.reset()

In [3]:
df_data=pd.read_csv("DataSetNormalised.csv")

In [4]:
df_label=pd.read_csv("LabelSet.csv")

df_data

In [5]:
x = np.array(df_data.iloc[:,1:])
y = np.array(df_label.iloc[:,1])

In [6]:
x = x.astype(np.float32)

In [10]:
data = {}
acts = ["gelu"]
x_train = x[:][:250000]
y_train = y[:][:250000]
x_alpha = x[:][10000000:]
y_alpha = y[:][10000000:]


print(f"Doing for '{act}'")
d_rate = 0 #The rate of droupout for the layers in the ResNet
act='gelu'#The activation function use ('relu','swish','elu','gelu','selu')
lr = 0.01 #The default learning rate Default 0.01
mom = 0.9 #The momentum parameter Default 0.9
test_split = 1 #Default 1 and range[1,0]
#Reducing test_split will make the model faster but will also reduce the accuracy of the Validation metric
#On changing test_split run the lower initialisation cell, marked below**

t1 = time()
#On changing test_split run this--------**
size = test_split*100000

x_test = x_alpha[:][:int(size)]
y_test = y_alpha[:][:int(size)]


def identity_block(X,no_nodes,Activation):#A custom Residual identity block with a skip connection
    X_shortcut=X
    X=tf.keras.layers.Dense(no_nodes,activation=Activation)(X)
    X=tf.keras.layers.BatchNormalization(axis=1)(X)
    X=tf.keras.layers.Dropout(d_rate)(X)
    X=tf.keras.layers.Dense(no_nodes,activation=Activation)(X)
    X=tf.keras.layers.BatchNormalization(axis=1)(X)
    X=tf.keras.layers.Dropout(d_rate)(X)
    X=tf.keras.layers.Dense(no_nodes,activation=Activation)(X)
    X=tf.keras.layers.Dropout(d_rate)(X)
    X= tf.keras.layers.Add()([X_shortcut, X])
    X=tf.keras.layers.Activation('swish')(X)
    X_shortcut2=X
    X=tf.keras.layers.Dense(no_nodes,activation=Activation)(X)
    X=tf.keras.layers.BatchNormalization(axis=1)(X)
    X=tf.keras.layers.Dropout(d_rate)(X)
    X=tf.keras.layers.Dense(no_nodes,activation=Activation)(X)
    X=tf.keras.layers.BatchNormalization(axis=1)(X)
    X=tf.keras.layers.Dropout(d_rate)(X)
    X=tf.keras.layers.Dense(no_nodes,activation=Activation)(X)
    X=tf.keras.layers.Dropout(d_rate)(X)
    X= tf.keras.layers.Add()([X_shortcut, X, X_shortcut2])
    X=tf.keras.layers.Activation('swish')(X)
    return X


inputs=tf.keras.layers.Input(shape=(28,))                    # Keras input layer
X=tf.keras.layers.Dense(300,activation=act)(inputs)
X=identity_block(X=X,no_nodes=300,Activation=act)
X=identity_block(X=X,no_nodes=300,Activation=act)
X=identity_block(X=X,no_nodes=300,Activation=act)
X=identity_block(X=X,no_nodes=300,Activation=act)
X=identity_block(X=X,no_nodes=300,Activation=act)        
outputs=tf.keras.layers.Dense(1,activation='sigmoid')(X)     # keras output layer
model=tf.keras.Model(inputs,outputs)                         # Model declaration


model.compile(
        optimizer=tf.keras.optimizers.Adam(
        learning_rate=lr,
        beta_1=mom,
        beta_2=0.999),
        loss='binary_crossentropy',
        metrics=['accuracy','AUC']
    )

# history = History()

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                             factor=0.2,
                             patience=5,
                             min_lr=0.00001)

t2 = time()

model.fit(
    x_train,
    y_train,
    epochs=100,
    batch_size=10240*2,
    validation_data=(x_test,y_test),
    callbacks=[reduce_lr],
    verbose = True
)

t3 = time()

data[act] = history.history
data[act]["time"] = [(t2-t1)//0.001, (t3-t2)//0.001]

print("\n\n")
# mem_clr()

Doing for 'gelu'
Epoch 1/100
13/13 [==============================] - 13s 502ms/step - loss: 31.2057 - accuracy: 0.5052 - auc: 0.5025 - val_loss: 77799.8359 - val_accuracy: 0.5282 - val_auc: 0.4996 - lr: 0.0100
Epoch 2/100
13/13 [==============================] - 6s 437ms/step - loss: 13.7291 - accuracy: 0.5054 - auc: 0.5022 - val_loss: 821.0826 - val_accuracy: 0.4736 - val_auc: 0.4974 - lr: 0.0100
Epoch 3/100
13/13 [==============================] - 6s 436ms/step - loss: 6.3132 - accuracy: 0.5072 - auc: 0.5075 - val_loss: 8.1223 - val_accuracy: 0.5294 - val_auc: 0.5084 - lr: 0.0100
Epoch 4/100
13/13 [==============================] - 6s 436ms/step - loss: 2.2822 - accuracy: 0.5099 - auc: 0.5133 - val_loss: 1.3435 - val_accuracy: 0.5317 - val_auc: 0.5125 - lr: 0.0100
Epoch 5/100
13/13 [==============================] - 6s 437ms/step - loss: 1.1884 - accuracy: 0.5161 - auc: 0.5171 - val_loss: 0.9317 - val_accuracy: 0.5321 - val_auc: 0.5107 - lr: 0.0100
Epoch 6/100
13/13 [===============

Epoch 44/100
13/13 [==============================] - 6s 435ms/step - loss: 0.6759 - accuracy: 0.5858 - auc: 0.6115 - val_loss: 0.6794 - val_accuracy: 0.5739 - val_auc: 0.5900 - lr: 4.0000e-04
Epoch 45/100
13/13 [==============================] - 6s 436ms/step - loss: 0.6758 - accuracy: 0.5868 - auc: 0.6120 - val_loss: 0.6798 - val_accuracy: 0.5744 - val_auc: 0.5910 - lr: 4.0000e-04
Epoch 46/100
13/13 [==============================] - 6s 435ms/step - loss: 0.6763 - accuracy: 0.5866 - auc: 0.6128 - val_loss: 0.6800 - val_accuracy: 0.5751 - val_auc: 0.5914 - lr: 4.0000e-04
Epoch 47/100
13/13 [==============================] - 6s 437ms/step - loss: 0.6728 - accuracy: 0.5885 - auc: 0.6148 - val_loss: 0.6788 - val_accuracy: 0.5752 - val_auc: 0.5928 - lr: 4.0000e-04
Epoch 48/100
13/13 [==============================] - 6s 436ms/step - loss: 0.6727 - accuracy: 0.5882 - auc: 0.6157 - val_loss: 0.6780 - val_accuracy: 0.5763 - val_auc: 0.5940 - lr: 4.0000e-04
Epoch 49/100
13/13 [===============

In [30]:
pprint(data2)

{}


In [31]:
for key, value in data.items():
    print(f"max = {max(value['accuracy'])}, last = {value['accuracy'][-1]} ({key})")
# winner = 'gelu'
    
print("\n")

for key, value in data.items():
    print(f"max = {max(value['auc'])}, last = {value['auc'][-1]} ({key})")
# winner = 'gelu'
    
print("\n")

for key, value in data.items():
    print(f"time = {value['time']} ({key})")
# winner = 'elu'

max = 0.5373200178146362, last = 0.5294319987297058 (relu)
max = 0.5374559760093689, last = 0.5374559760093689 (swish)
max = 0.5223199725151062, last = 0.5223199725151062 (elu)
max = 0.5484480261802673, last = 0.5413039922714233 (gelu)
max = 0.5214719772338867, last = 0.5162439942359924 (selu)


max = 0.5494561195373535, last = 0.5416449904441833 (relu)
max = 0.5556849241256714, last = 0.5556849241256714 (swish)
max = 0.5259912610054016, last = 0.5235568881034851 (elu)
max = 0.5593453049659729, last = 0.5559259653091431 (gelu)
max = 0.5246537923812866, last = 0.5216187238693237 (selu)


time = [0.46911120414733887, 42.470707178115845] (relu)
time = [0.5133793354034424, 50.65196919441223] (swish)
time = [0.5508296489715576, 43.089529275894165] (elu)
time = [560.0, 56999.0] (gelu)
time = [0.4726884365081787, 42.457555055618286] (selu)


d_rate=0 #The rate of droupout for the layers in the ResNet
act='swish'#The activation function use ('relu','swish','elu','gelu','selu')
lr=0.01 #The default learning rate Default 0.01
mom=0.9 #The momentum parameter Default 0.9
test_split=1 #Default 1 and range[1,0]
#Reducing test_split will make the model faster but will also reduce the accuracy of the Validation metric
#On changing test_split run the lower initialisation cell, marked below**

#On changing test_split run this--------**
size=10000000 + test_split*1000000
x_train=x[:][:10000000]
y_train=y[:][:10000000]
x_test=x[:][10000000:size]
y_test=y[:][10000000:size]

y_train.shape

def identity_block(X,no_nodes,Activation):#A custom Residual identity block with a skip connection
    X_shortcut=X
    #There will be 3 layers in total for the sequential connection
    #Activation will be specified by the user as per the Keras Documentation
    #no_nodes denote the size of the model which will be built
    #Applying Batch Normalisation to size the inputs
    #Layer 1 -----------------------------------------------
    X=tf.keras.layers.Dense(no_nodes,activation=Activation)(X)
    X=tf.keras.layers.BatchNormalization(axis=1)(X)
    X=tf.keras.layers.Dropout(d_rate)(X)
    #--------------------------------------------------------
    #Layer 2-------------------------------------------------
    X=tf.keras.layers.Dense(no_nodes,activation=Activation)(X)
    X=tf.keras.layers.BatchNormalization(axis=1)(X)
    X=tf.keras.layers.Dropout(d_rate)(X)
    #--------------------------------------------------------
    #Layer 3-------------------------------------------------
    X=tf.keras.layers.Dense(no_nodes,activation=Activation)(X)
    X=tf.keras.layers.Dropout(d_rate)(X)
    #--------------------------------------------------------
    #Layer 4-------------------------------------------------
    X= tf.keras.layers.Add()([X_shortcut, X])
    X=tf.keras.layers.Activation('swish')(X)
    #--------------------------------------------------------
    X_shortcut2=X
     #Layer 5 -----------------------------------------------
    X=tf.keras.layers.Dense(no_nodes,activation=Activation)(X)
    X=tf.keras.layers.BatchNormalization(axis=1)(X)
    X=tf.keras.layers.Dropout(d_rate)(X)
    #--------------------------------------------------------
    #Layer 6-------------------------------------------------
    X=tf.keras.layers.Dense(no_nodes,activation=Activation)(X)
    X=tf.keras.layers.BatchNormalization(axis=1)(X)
    X=tf.keras.layers.Dropout(d_rate)(X)
    #--------------------------------------------------------
    #Layer 7-------------------------------------------------
    X=tf.keras.layers.Dense(no_nodes,activation=Activation)(X)
    X=tf.keras.layers.Dropout(d_rate)(X)
    #--------------------------------------------------------
    X= tf.keras.layers.Add()([X_shortcut, X,X_shortcut2])
    #Layer 8-------------------------------------------------
    X=tf.keras.layers.Activation('swish')(X)
    return X
    

inputs=tf.keras.layers.Input(shape=(28,))                    #Keras input layer
X=tf.keras.layers.Dense(300,activation=act)(inputs)
X=identity_block(X=X,no_nodes=300,Activation=act)
X=identity_block(X=X,no_nodes=300,Activation=act)
X=identity_block(X=X,no_nodes=300,Activation=act)
X=identity_block(X=X,no_nodes=300,Activation=act)
X=identity_block(X=X,no_nodes=300,Activation=act)        
outputs=tf.keras.layers.Dense(1,activation='sigmoid')(X)     # keras output layer
model=tf.keras.Model(inputs,outputs)                         # Model declaration

model.summary()

model.compile(
        optimizer=tf.keras.optimizers.Adam(
        learning_rate=lr,
        beta_1=mom,
        beta_2=0.999),
        loss='binary_crossentropy',
        metrics=['accuracy','AUC']
    )

model.fit(
    x_train,
    y_train,
    epochs=1,
    batch_size=10240,
    validation_data=(x_test,y_test),
    callbacks=[history]    
)

print(history.history)

print(history.history)